# Exploring and Analyzing MrBeast YouTube Comments



YouTube is not only a popular social media platform but is also the second largest search engine. Consequently, promotional content is a main staple on YouTube, reaching a very large audience. Videos are accompanied by a comments section which allow viewers to engage with content creators or other fans. With this level of engagement, companies flock to larger channels to offer sponsorships to promote products within a video in exchange for benefits.   

Of the many YouTube content creators, MrBeast is known for his unique expenditures and charitable acts, a lot of which are sponsored by companies which provide the capital. I am interested in analyzing YouTube comments on the MrBeast channel since his content is not controversial and I am curious to see if the comments are overwhelmingly positive as rumored. MrBeast often films with his good friends and I want to see who is the most popular across the videos (I suspect Chandler). Similarly, I want to see the most defining words the channel.

Moreover, to get the comments, I have to build and clean the dataset from scratch which is something I want to challenge myself to do. I can use this exposure to scraping to improve existing datasets or create new datasets to address business problems. 



#### Import Libraries

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

!pip install emoji
from emoji import UNICODE_EMOJI

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

!pip install TextBlob
from textblob import TextBlob
from textblob import Word

In [12]:
file_mrbeast = 'MrBeast_comments.csv'
df = pd.read_csv(file_mrbeast)
df.head()

,Unnamed: 0,comment,comment_id,author_url,author_name,reply_count,like_count,date,vidid,total_reply_counts,vid_title,just_date
0,0,"""I'm sorry, international fans""\n\n*Cries in P...",UgxTQx4UK1GxX8csGQN4AaABAg,http://www.youtube.com/channel/UCRvvUJye_0IPDE...,Allona Shanley Magpantay,0,0,2021-01-13 16:59:52+00:00,dg2Ag3e8W-Q,0,I Opened A Restaurant That Pays You To Eat At It,2021-01-13
1,1,Relatable,UgxU3cNgqkyGMDAuXxJ4AaABAg,http://www.youtube.com/channel/UC8FTumc39Z00Vn...,Sagar Gwal,0,0,2021-01-13 16:59:46+00:00,LeYsRMZFUq0,0,"I Gave People $1,000,000 But ONLY 1 Minute To ...",2021-01-13
2,2,How do you Gus love that crap,UgyUaDvKrt8ioZ7QTbh4AaABAg,http://www.youtube.com/channel/UCnF6pvaO6iYSnY...,Noah Rodriguez,0,0,2021-01-13 16:59:43+00:00,A-vX1AGBGsc,0,I Went Back To 1st Grade For A Day,2021-01-13
3,3,I wouldn’t be surprised if MrBeast became one ...,UgwBA9pqqOVRBKXl17B4AaABAg,http://www.youtube.com/channel/UCOQ-Z0f0gWPtGQ...,TheFlaaa,0,0,2021-01-13 16:59:43+00:00,5Fg9oZk-5uA,0,I Bought Everything In 5 Stores,2021-01-13
4,4,Chandler:eats a rose,UgxhD7F8V6WUiYXvA5F4AaABAg,http://www.youtube.com/channel/UCZocmKEhNiRotw...,Emilija Mikuckyte,0,0,2021-01-13 16:59:43+00:00,i6OUv48w1bk,0,"Surprising My Girlfriend With 100,000 Roses Fo...",2021-01-13


In [13]:
# Remove comments by the author of the video i.e. MrBeast
comments = df.query('author_name != "MrBeast"')
comments.head()

,Unnamed: 0,comment,comment_id,author_url,author_name,reply_count,like_count,date,vidid,total_reply_counts,vid_title,just_date
0,0,"""I'm sorry, international fans""\n\n*Cries in P...",UgxTQx4UK1GxX8csGQN4AaABAg,http://www.youtube.com/channel/UCRvvUJye_0IPDE...,Allona Shanley Magpantay,0,0,2021-01-13 16:59:52+00:00,dg2Ag3e8W-Q,0,I Opened A Restaurant That Pays You To Eat At It,2021-01-13
1,1,Relatable,UgxU3cNgqkyGMDAuXxJ4AaABAg,http://www.youtube.com/channel/UC8FTumc39Z00Vn...,Sagar Gwal,0,0,2021-01-13 16:59:46+00:00,LeYsRMZFUq0,0,"I Gave People $1,000,000 But ONLY 1 Minute To ...",2021-01-13
2,2,How do you Gus love that crap,UgyUaDvKrt8ioZ7QTbh4AaABAg,http://www.youtube.com/channel/UCnF6pvaO6iYSnY...,Noah Rodriguez,0,0,2021-01-13 16:59:43+00:00,A-vX1AGBGsc,0,I Went Back To 1st Grade For A Day,2021-01-13
3,3,I wouldn’t be surprised if MrBeast became one ...,UgwBA9pqqOVRBKXl17B4AaABAg,http://www.youtube.com/channel/UCOQ-Z0f0gWPtGQ...,TheFlaaa,0,0,2021-01-13 16:59:43+00:00,5Fg9oZk-5uA,0,I Bought Everything In 5 Stores,2021-01-13
4,4,Chandler:eats a rose,UgxhD7F8V6WUiYXvA5F4AaABAg,http://www.youtube.com/channel/UCZocmKEhNiRotw...,Emilija Mikuckyte,0,0,2021-01-13 16:59:43+00:00,i6OUv48w1bk,0,"Surprising My Girlfriend With 100,000 Roses Fo...",2021-01-13


In [14]:
def removeNewline(comment):
  return comment.str.replace("\n", " ")

In [15]:
comments['comment'] = removeNewline(comments['comment'])

In [16]:
comments.head()

,Unnamed: 0,comment,comment_id,author_url,author_name,reply_count,like_count,date,vidid,total_reply_counts,vid_title,just_date
0,0,"""I'm sorry, international fans"" *Cries in Phi...",UgxTQx4UK1GxX8csGQN4AaABAg,http://www.youtube.com/channel/UCRvvUJye_0IPDE...,Allona Shanley Magpantay,0,0,2021-01-13 16:59:52+00:00,dg2Ag3e8W-Q,0,I Opened A Restaurant That Pays You To Eat At It,2021-01-13
1,1,Relatable,UgxU3cNgqkyGMDAuXxJ4AaABAg,http://www.youtube.com/channel/UC8FTumc39Z00Vn...,Sagar Gwal,0,0,2021-01-13 16:59:46+00:00,LeYsRMZFUq0,0,"I Gave People $1,000,000 But ONLY 1 Minute To ...",2021-01-13
2,2,How do you Gus love that crap,UgyUaDvKrt8ioZ7QTbh4AaABAg,http://www.youtube.com/channel/UCnF6pvaO6iYSnY...,Noah Rodriguez,0,0,2021-01-13 16:59:43+00:00,A-vX1AGBGsc,0,I Went Back To 1st Grade For A Day,2021-01-13
3,3,I wouldn’t be surprised if MrBeast became one ...,UgwBA9pqqOVRBKXl17B4AaABAg,http://www.youtube.com/channel/UCOQ-Z0f0gWPtGQ...,TheFlaaa,0,0,2021-01-13 16:59:43+00:00,5Fg9oZk-5uA,0,I Bought Everything In 5 Stores,2021-01-13
4,4,Chandler:eats a rose,UgxhD7F8V6WUiYXvA5F4AaABAg,http://www.youtube.com/channel/UCZocmKEhNiRotw...,Emilija Mikuckyte,0,0,2021-01-13 16:59:43+00:00,i6OUv48w1bk,0,"Surprising My Girlfriend With 100,000 Roses Fo...",2021-01-13


#### Vader Sentiments

VADER (Valence Aware Dictionary for Sentiment Reasoning) is a model that is sensitive to both polarity and intensity of emotion. It relies on a dictionary that maps lexical feactures to emotion intensities as sentiment scores. The sum of the intensities of each word in text is the sentiment score. Here, I am using polarity classification since I only care if the text expresses a positive or negative opinion but not the degree of how positive or negative the text is. VADER can also tell something like "did not love" as negative when compared to just "love."

In [21]:
sentiment = SentimentIntensityAnalyzer()
sentiment.polarity_scores(comments.comment[0])
comments['vader_sentiment'] = comments.comment.apply(lambda x: sentiment.polarity_scores(x))
comments['vader_neg_sentiment'] = comments.vader_sentiment.apply(lambda x: x['neg'])
comments['vader_pos_sentiment'] = comments.vader_sentiment.apply(lambda x: x['pos'])
comments['vader_neu_sentiment'] = comments.vader_sentiment.apply(lambda x: x['neu'])
comments['vader_comp_sentiment'] = comments.vader_sentiment.apply(lambda x: x['compound'])

Positive sentiment

In [22]:
comments.sort_values(by=['vader_comp_sentiment'], ascending=False)[['comment']].head()

,comment
5567,This made my day 😊😊😊😊😊😊😊😊😊☺️☺️☺️😊😊😊😊😊😊😊😊😊😊😊😊😊😊...
1325,When the guy go to the table and he jumped and...
4082,❤️❤️❤️😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍❤️❤️❤️❤️❤️❤️❤️...
6797,Im from Philippines i hipe mr beast visit me h...
6555,😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😔😂😂😂😂😂


Neutral sentiment

In [26]:
vader_neutral = comments.query('vader_comp_sentiment == 0').rename(columns={"comment": "vader_comment"})[['vader_comment']].reset_index(drop=True)
vader_neutral

,vader_comment
0,Relatable
1,Chandler:eats a rose
2,I SAW MR.BEAST GET TO 51 MILLION!
3,Buy a franchise
4,Iv heard people say you shouldnt donate to sal...
...,...
5279,Is it just me who was watching the football in...
5280,How do you have so much money
5281,Elise Stefanik
5282,Ryan did the spoon flip


Negative sentiment

In [20]:
comments.sort_values(by=['vader_comp_sentiment'], ascending=True)[['comment']].head()

,comment
8743,😥😥😥😥😥😥😢😢😢😢😢😢😢😢😢😢😢😱😱😱
3022,PEWDIEDIE AAAAAAAAH 💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥💥!!!!!!...
8511,So sad for chandler 😭😭😭😭😭😭😭😭
5617,My reaction: *H* 😄😥😫😓🤤😝😤😞😖😲🤑😵🥴🤯😨😭😱🥺 *Fricc...
2493,I feel so bad for chandler 😢😢😢😢😢😢


This is more like pity... Let's try something else.

#### Textblob Sentiments

In [23]:
comments['textblob_polarity'] = comments['comment'].map(lambda text: TextBlob(text).sentiment.polarity)
comments['comment_len'] = comments['comment'].astype(str).apply(len)
comments['word_count'] = comments['comment'].apply(lambda x: len(str(x).split()))

Positive sentiment

In [24]:
comments.sort_values(by=['textblob_polarity'], ascending=False)[['comment', 'textblob_polarity']].head()

,comment,textblob_polarity
8705,Teacher: THE BEST JOB IN TH WORLD IS WHAT? ST...,1.0
6961,4:05 he says microwaves are the best but look ...,1.0
2835,Best channel ever,1.0
3163,You have a lot of fans from Croatia <3,1.0
4336,Help me. Please donate me some money. Perfect ...,1.0


Neutral sentiment

In [28]:
textblob_neutral = comments.query('textblob_polarity == 0').rename(columns={"comment": "textblob_comment"})[['textblob_comment']].reset_index(drop=True)
textblob_neutral

,textblob_comment
0,Relatable
1,I SAW MR.BEAST GET TO 51 MILLION!
2,Buy a franchise
3,Iv heard people say you shouldnt donate to sal...
4,*what if he goes bankrupt*
...,...
6770,Elise Stefanik
6771,Ryan did the spoon flip
6772,why am i just now seeing this on my recommended
6773,Chan Chan's Dog


Negative sentiment

In [25]:
comments.sort_values(by=['textblob_polarity'], ascending=True)[['comment', 'textblob_polarity']].head()

,comment,textblob_polarity
6674,this is insane aaa,-1.0
6869,How dare you guys use lightning McQueens cars ...,-1.0
6858,im getting insane watching,-1.0
1074,"Thats insane man, he now has cameramans. Becau...",-1.0
3905,Its terrifying,-1.0
